In [1]:
import cdsapi
import zipfile
import xarray as xr 
import rioxarray as rio 
import os
from tqdm import tqdm
import pandas as pd
from datetime import datetime as dt
import datetime
import rasterio
from rasterio.plot import show
from tqdm import tqdm
import numpy as np
import schedule
from netCDF4 import Dataset

import netCDF4 as nc
import os
import pathlib as Path
import xarray as xr
import cdsapi
import glob

import pandas as pd
from osgeo import gdal

import netCDF4 as nc
import os
import pathlib as Path
import xarray as xr
import cdsapi

import datetime as dt

import geopandas as gpd

/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_188791/3976756483.py:35: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_

# Download

In [2]:
def download_AgERA5_month(selected_area, variables, query_date):

    print("===== download_AgERA5_month =====")

    try:
        query_year = query_date
        #query_month = query_date.strftime('%m')

        #print("Today is",query_date.date(),"; last available date on AgERA5 is",query_date.date())

        c = cdsapi.Client()

        if not os.path.exists('0_downloads/'):
            os.makedirs('0_downloads/')

        for variable in variables :
            
            print("\nDownloading values for variable",variable,"/",query_year)

            zip_path = '0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'

            request = {
                    'format': 'zip',
                    'day': [
                        '01', '02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                        '13', '14', '15',
                        '16', '17', '18',
                        '19', '20', '21',
                        '22', '23', '24',
                        '25', '26', '27',
                        '28', '29', '30',
                        '31',
                    ],
                    'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11','12'
                    ],
                    'year': query_year,
                    'variable': variable[0],
                    'statistic': variable[1],
                    'area': area[selected_area],
                }

            # la requête doit être adaptée pour cette variable
            if variable[0] == "solar_radiation_flux" :
                del request["statistic"]

            if variable[0] == "precipitation_flux" :
                del request["statistic"]
                
                
            c.retrieve(
                'sis-agrometeorological-indicators',
                request,
                zip_path)

            print("Download OK")

    except :
        print("/!\ Download NOT OK")

# Extraction

In [3]:
def extract_agERA5_month(selected_area, variables, query_date):

    print("===== extract_agERA5_month =====")

    try:
        query_year = query_date

        for variable in tqdm(variables) :

            zip_path = '0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'
            extraction_path = '1_extraction/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'/'

            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extraction_path)
            except:
                pass
            
            aggregate_path = '2_aggregation/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'/'
            
        print("Extraction OK")
        
        

    except :
        print("/!\ Extraction NOT OK")

# Site extraction

In [4]:
os.listdir()

['.ipynb_checkpoints',
 '0_downloads',
 '1_extraction',
 'get_AgERA5_data_download_Rwanda.ipynb',
 'RV_project_area_final.cpg',
 'RV_project_area_final.dbf',
 'RV_project_area_final.prj',
 'RV_project_area_final.qmd',
 'RV_project_area_final.shp',
 'RV_project_area_final.shx']

In [5]:
aoi = gpd.read_file('RV_project_area_final.shp')

# Run all

In [6]:
first = pd.date_range(start='2020', end='2023', freq='YS')
datelist = [i.strftime('%Y') for i in first]

In [7]:
n, e, s, w =  aoi.total_bounds[-1], aoi.total_bounds[-2], aoi.total_bounds[1], aoi.total_bounds[2]

In [8]:
aoi.total_bounds

array([ 32.63280385, -18.79800503,  33.06717812, -17.50311559])

In [9]:
str(n.round(2))

'-17.5'

In [10]:
n, e, s, w

(-17.503115589999936,
 33.067178124000066,
 -18.79800503399997,
 33.067178124000066)

In [11]:
n, e, s, w =  aoi.total_bounds[-1], aoi.total_bounds[-2], aoi.total_bounds[1], aoi.total_bounds[2]

# parameters
for year in datelist:
    query_date = year
    area = {'Zimbabwe': [
            -17.5, 32.63, -18.79,
            33.06]}
    selected_area = "Zimbabwe"
    variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily")]
    download_AgERA5_month(selected_area, variables, query_date)
    extract_agERA5_month(selected_area, variables, query_date)
    

#calculate_AgERA5_ET0_and_save(selected_area, variables, query_date)
#print("===== Query date",query_date.,"all done ! =====")

===== download_AgERA5_month =====



/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:31:58,998 INFO Welcome to the CDS
2023-08-30 06:31:58,999 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:31:59,210 INFO Request is completed
2023-08-30 06:31:59,211 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-comp

Download OK



/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:32:03,784 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data9/dataset-sis-agrometeorological-indicators-272761ca-6e9d-4a85-ba79-b06e9f560402.zip to 0_downloads/AgERA5_Zimbabwe_2m_temperature_24_hour_maximum_2020.zip (8.3M)
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download-0009-clone.copernicus-climate.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:32:06,651 INFO Down

Download OK



...

Download OK



/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:41:02,510 INFO Request is queued
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2023-08-30 06:41:03,689 INFO Request is running
/home/c4ubuntu/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised.

Download OK
===== extract_agERA5_month =====


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:58<00:00, 19.34s/it]

Extraction OK


# Aggregate

### Daily

In [12]:
variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily")]

selected_area = 'Zimbabwe'

first = pd.date_range(start='2020', end='2023', freq='YS')
datelist = [i.strftime('%Y') for i in first]

for year in datelist:
    for variable in variables:      
        files = glob.glob(os.path.join('1_extraction','*{}_{}_{}_{}'.format(selected_area,variable[0],variable[1],year),'*'))
        df = xr.open_mfdataset(files, parallel=True, engine="netcdf4", decode_times=True, chunks={'time': 300},
                       data_vars='minimal', coords='minimal', compat='override')
        if not os.path.exists('2_aggregate/'):
            os.makedirs('2_aggregate/')
        print('Writing {} {}'.format(variable,year))
        df.to_netcdf('2_aggregate/AgERA5_{}_{}_{}_{}.nc'.format(selected_area,variable[0],variable[1],year), mode='w')
        df.close()
        print('Finished writing {} {}'.format(variable, year))

Writing ('2m_temperature', '24_hour_minimum') 2020
Finished writing ('2m_temperature', '24_hour_minimum') 2020
Writing ('2m_temperature', '24_hour_maximum') 2020
Finished writing ('2m_temperature', '24_hour_maximum') 2020
Writing ('precipitation_flux', 'daily') 2020
Finished writing ('precipitation_flux', 'daily') 2020
Writing ('2m_temperature', '24_hour_minimum') 2021
Finished writing ('2m_temperature', '24_hour_minimum') 2021
Writing ('2m_temperature', '24_hour_maximum') 2021
Finished writing ('2m_temperature', '24_hour_maximum') 2021
Writing ('precipitation_flux', 'daily') 2021
Finished writing ('precipitation_flux', 'daily') 2021
Writing ('2m_temperature', '24_hour_minimum') 2022
Finished writing ('2m_temperature', '24_hour_minimum') 2022
Writing ('2m_temperature', '24_hour_maximum') 2022
Finished writing ('2m_temperature', '24_hour_maximum') 2022
Writing ('precipitation_flux', 'daily') 2022
Finished writing ('precipitation_flux', 'daily') 2022
Writing ('2m_temperature', '24_hour_m

### Annual

In [13]:
variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("precipitation_flux","daily")]

selected_area = 'Zimbabwe'

for variable in variables:      
    files = glob.glob(os.path.join('2_aggregate','*{}_{}_{}*'.format(selected_area,variable[0],variable[1])))
    df = xr.open_mfdataset(files, parallel=True, engine="netcdf4", decode_times=True, chunks={'time': 1000},
                       data_vars='minimal', coords='minimal', compat='override')
    if not os.path.exists('3_combined'):
        os.makedirs('3_combined')
    print('Writing {}'.format(variable))
    df.to_netcdf('3_combined/AgERA5_{}_{}_{}.nc'.format(selected_area,variable[0],variable[1]), mode='w')
    df.close()
    print('Finished writing {}'.format(variable))

Writing ('2m_temperature', '24_hour_minimum')
Finished writing ('2m_temperature', '24_hour_minimum')
Writing ('2m_temperature', '24_hour_maximum')
Finished writing ('2m_temperature', '24_hour_maximum')
Writing ('precipitation_flux', 'daily')
Finished writing ('precipitation_flux', 'daily')


## Make sitefiles

In [14]:
signifdigits=3
data_type = 'AgERA5'
country = 'Zimbabwe'
outdir= f'/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe/'

In [15]:
variables = []
for file in glob.glob(os.path.join('3_combined',f'AgERA5_{country}*')):
    x = os.path.basename(file).split('_Zimbabwe_')[-1].replace('.nc', '')
    variables.append(x)
variables = np.unique(variables)

In [16]:
variables

array(['2m_temperature_24_hour_maximum', '2m_temperature_24_hour_minimum',
       'precipitation_flux_daily'], dtype='<U30')

In [17]:
# Get the file names
tmin_file = glob.glob(os.path.join('3_combined',f'AgERA5_{country}_{variables[1]}*'))[0]
tmax_file = glob.glob(os.path.join('3_combined',f'AgERA5_{country}_{variables[0]}*'))[0]
pr_file = glob.glob(os.path.join('3_combined',f'AgERA5_{country}_{variables[2]}*'))[0]

In [18]:
#Read the datasets
ds = xr.open_dataset(tmin_file)
ds1 = xr.open_dataset(tmax_file)
ds2 = xr.open_dataset(pr_file)

In [19]:
for year in np.unique(ds.time.dt.year.values):
    year_ds = ds.where(ds.time.dt.year == year, drop=True)
    year_ds1 = ds1.where(ds1.time.dt.year == year, drop=True)
    year_ds2 = ds2.where(ds2.time.dt.year == year, drop=True)
    
    year_ds['time'] = year_ds.time.dt.dayofyear
    year_ds1['time'] = year_ds1.time.dt.dayofyear
    year_ds2['time'] = year_ds2.time.dt.dayofyear
    
    dates = year_ds.time.astype(str).values
    signifdigits=3
    
    tasmin=year_ds["Temperature_Air_2m_Min_24h"]-273.15
    tasmax=year_ds1["Temperature_Air_2m_Max_24h"]-273.15
    pr=year_ds2["Precipitation_Flux"]
    
    data_type = 'AgERA5'

    #iterating through latitudes and longitudes
    for lat in ds.lat.data:
        for lon in ds.lon.data:
            latstring=np.round(lat.data,2)
            lonstring=np.round(lon.data,2)
            header = f'Station_{latstring}_{lonstring}\n'
            outfilename="{}_{}_{}_{}_{}.csv".format(data_type,country,latstring,lonstring, year)
            outfile="{}/{}".format(outdir,outfilename)
            #data for given lat and lon
            site_tasmin=np.round(tasmin.sel(lat=lat, lon=lon).data,signifdigits)
            site_tasmax=np.round(tasmax.sel(lat=lat, lon=lon).data,signifdigits)
            site_pr=np.round(pr.sel(lat=lat, lon=lon).data,signifdigits)
            if np.count_nonzero(np.isnan(site_tasmax)) == 0:  
            #opening file for writing
                with open(outfile, "w") as outf:
                    outf.write(header)
                #iterating through time steps
                    for i in range(site_tasmin.shape[0]-1):
                        line=f"{dates[i]}\t%.2f\t%.2f\t%.2f\n"%(site_tasmax[i],site_tasmin[i],
                                                                            site_pr[i])
                        outf.write(line)
                print(outfile)

/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.7_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.8_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.9_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_33.0_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.7_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.8_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.9_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_33.0_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.7_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.8_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.9_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_33.0_2020.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbab

/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_33.0_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.7_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.8_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.9_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_33.0_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.9_32.7_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.9_32.8_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.9_32.9_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.9_33.0_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-18.0_32.7_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-18.0_32.8_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-18.0_32.9_2022.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbab

In [20]:
ds = ds.groupby('time.dayofyear').mean(dim='time')
ds1 = ds1.groupby('time.dayofyear').mean(dim='time')
ds2 = ds2.groupby('time.dayofyear').mean(dim='time')

In [21]:
dates = ds.dayofyear.astype(str).values
signifdigits=3

In [22]:
tasmin=ds["Temperature_Air_2m_Min_24h"]-273.15
tasmax=ds1["Temperature_Air_2m_Max_24h"]-273.15
pr=ds2["Precipitation_Flux"]

In [23]:
tasmin.shape[0]-1

365

In [24]:
data_type = 'AgERA5'

#iterating through latitudes and longitudes
for lat in ds.lat.data:
    for lon in ds.lon.data:
        latstring=np.round(lat.data,2)
        lonstring=np.round(lon.data,2)
        header = f'Station_{latstring}_{lonstring}\n'
        outfilename="{}_{}_{}_{}.csv".format(data_type,country,latstring,lonstring)
        outfile="{}/{}".format(outdir,outfilename)
        #data for given lat and lon
        site_tasmin=np.round(tasmin.sel(lat=lat, lon=lon).data,signifdigits)
        site_tasmax=np.round(tasmax.sel(lat=lat, lon=lon).data,signifdigits)
        site_pr=np.round(pr.sel(lat=lat, lon=lon).data,signifdigits)
        if np.count_nonzero(np.isnan(site_tasmax)) == 0:  
        #opening file for writing
            with open(outfile, "w") as outf:
                outf.write(header)
            #iterating through time steps
                for i in range(site_tasmin.shape[0]-1):
                    line=f"{dates[i]}\t%.2f\t%.2f\t%.2f\n"%(site_tasmax[i],site_tasmin[i],
                                                                        site_pr[i])
                    outf.write(line)
            print(outfile)

/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.7.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.8.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_32.9.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.6_33.0.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.7.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.8.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_32.9.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.7_33.0.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.7.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.8.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_32.9.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.8_33.0.csv
/home/c4ubuntu/c4gdata/RiftValley_Zimbabwe//AgERA5_Zimbabwe_-17.9_32.7.csv
/home/c4ubuntu/c4gdata/Ri